# 1. Package 설치

In [ ]:
%pip install --upgrade pymilvus langchain openai tiktoken

# 2. Knowledge Base 구성을 위한 데이터 생성

- 4. LangChain을 활용한 Vector Database 변경 (Chroma -> Milvus)과 동일함

In [10]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=128,
)

# 개인정보 보호법
document_name = './medical_with_markdown.docx'

loader = Docx2txtLoader(document_name)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [11]:
len_list = len(document_list)
len_list

86

In [12]:
document_list[len_list - 10]

Document(metadata={'source': './privacy_with_markdown.docx'}, page_content='1. 제25조제5항(제26조제8항에 따라 준용되는 경우를 포함한다)을 위반하여 고정형 영상정보처리기기의 설치 목적과 다른 목적으로 고정형 영상정보처리기기를 임의로 조작하거나 다른 곳을 비추는 자 또는 녹음기능을 사용한 자\n\n2. 제59조제1호를 위반하여 거짓이나 그 밖의 부정한 수단이나 방법으로 개인정보를 취득하거나 개인정보 처리에 관한 동의를 받는 행위를 한 자 및 그 사정을 알면서도 영리 또는 부정한 목적으로 개인정보를 제공받은 자\n\n3. 제60조를 위반하여 직무상 알게 된 비밀을 누설하거나 직무상 목적 외에 이용한 자\n\n\n\n제73조(벌칙) ① 다음 각 호의 어느 하나에 해당하는 자는 2년 이하의 징역 또는 2천만원 이하의 벌금에 처한다.\n\n1. 제36조제2항(제26조제8항에 따라 준용되는 경우를 포함한다)을 위반하여 정정ㆍ삭제 등 필요한 조치를 하지 아니하고 개인정보를 계속 이용하거나 이를 제3자에게 제공한 자\n\n2. 제37조제2항(제26조제8항에 따라 준용되는 경우를 포함한다)을 위반하여 개인정보의 처리를 정지하지 아니하고 개인정보를 계속 이용하거나 제3자에게 제공한 자\n\n3. 국내외에서 정당한 이유 없이 제39조의4에 따른 비밀유지명령을 위반한 자\n\n4. 제63조제1항(제26조제8항에 따라 준용되는 경우를 포함한다)에 따른 자료제출 요구에 대하여 법 위반사항을 은폐 또는 축소할 목적으로 자료제출을 거부하거나 거짓의 자료를 제출한 자\n\n5. 제63조제2항(제26조제8항에 따라 준용되는 경우를 포함한다)에 따른 출입ㆍ검사 시 자료의 은닉ㆍ폐기, 접근 거부 또는 위조ㆍ변조 등을 통하여 조사를 거부ㆍ방해 또는 기피한 자\n\n② 제1항제3호의 죄는 비밀유지명령을 신청한 자의 고소가 없으면 공소를 제기할 수 없다.\n\n[전문개정 2023. 3. 14.]')

In [13]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

True

In [25]:
import os

from langchain.vectorstores.zilliz import Zilliz

# 1. Set up the name of the collection to be created.
COLLECTION_NAME = 'medical_law_index'

# 2. Set up the dimension of the embeddings.
DIMENSION = 768

# 3. Set up the cohere api key
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# 4. Set up the connection parameters for your Zilliz Cloud cluster.
URI = os.environ["MILVUS_CLUSTER_ENDPOINT"]

# 5. Set up the token for your Zilliz Cloud cluster.
# You can either use an API key or a set of cluster username and password joined by a colon.
TOKEN = os.environ["MILVUS_TOKEN"]

In [26]:
# Connect using a MilvusClient object
from pymilvus import MilvusClient, DataType

# 1. Set up a Milvus client
client = MilvusClient(
    uri=URI,
    token=TOKEN
)

res = client.get_load_state(
    collection_name=COLLECTION_NAME
)
print(f"{COLLECTION_NAME} state: {res['state']}")
res_state = str(res['state'])

medical_law_index state: Loaded


In [28]:
is_create = False
if res_state == 'NotExist':
    is_create = True
print(f"is_create: {is_create}")

embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
connection_args = { 'uri': URI, 'token': TOKEN }

if is_create:
    ids = [str(i) for i in range(len(document_list))]

    vector_store = Zilliz(
        embedding_function=embeddings, 
        connection_args=connection_args,
        collection_name=COLLECTION_NAME,
        drop_old=True,
        auto_id=True
    ).from_documents(
        document_list,
        embedding=embeddings,
        collection_name=COLLECTION_NAME,
        connection_args=connection_args,
        ids = ids
    )
else:
    vector_store = Zilliz(
        embedding_function=embeddings,
        collection_name=COLLECTION_NAME,
        connection_args=connection_args
    )

is_create: False


In [29]:
query = '의료기관에서 의료사고가 발생했을 때, 병원과 의료진의 법적 책임은 어떻게 되나요?'

# 3. 답변 생성을 위한 Retrieval

- RetrievalQA에 전달하기 위해 retriever 생성
- search_kwargs의 k값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
- .invoke()를 호출해서 어떤 문서를 가져오는지 확인 가능

In [30]:
retriever = vector_store.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(metadata={'source': './medical_with_markdown.docx', 'pk': '4'}, page_content='③ 보건복지부장관은 제1항에 따라 전문병원으로 지정하는 경우 제2항 각 호의 사항 및 진료의 난이도 등에 대하여 평가를 실시하여야 한다.<개정 2010. 1. 18.>\n\n④ 보건복지부장관은 제1항에 따라 전문병원으로 지정받은 의료기관에 대하여 3년마다 제3항에 따른 평가를 실시하여 전문병원으로 재지정할 수 있다.<개정 2010. 1. 18., 2015. 1. 28.>\n\n⑤ 보건복지부장관은 제1항 또는 제4항에 따라 지정받거나 재지정받은 전문병원이 다음 각 호의 어느 하나에 해당하는 경우에는 그 지정 또는 재지정을 취소할 수 있다. 다만, 제1호에 해당하는 경우에는 그 지정 또는 재지정을 취소하여야 한다.<신설 2015. 1. 28.>\n\n1. 거짓이나 그 밖의 부정한 방법으로 지정 또는 재지정을 받은 경우\n\n2. 지정 또는 재지정의 취소를 원하는 경우\n\n3. 제4항에 따른 평가 결과 제2항 각 호의 요건을 갖추지 못한 것으로 확인된 경우\n\n⑥ 보건복지부장관은 제3항 및 제4항에 따른 평가업무를 관계 전문기관 또는 단체에 위탁할 수 있다.<개정 2010. 1. 18., 2015. 1. 28.>\n\n⑦ 전문병원 지정ㆍ재지정의 기준ㆍ절차 및 평가업무의 위탁 절차 등에 관하여 필요한 사항은 보건복지부령으로 정한다.<개정 2010. 1. 18., 2015. 1. 28.>\n\n[본조신설 2009. 1. 30.]\n\n\n\n제2장 의료인\n\n\n\n제1절 자격과 면허\n\n\n\n제4조(의료인과 의료기관의 장의 의무) ①의료인과 의료기관의 장은 의료의 질을 높이고 의료관련감염(의료기관 내에서 환자, 환자의 보호자, 의료인 또는 의료기관 종사자 등에게 발생하는 감염을 말한다. 이하 같다)을 예방하며 의료기술을 발전시키는 등 환자에게 최선의 의료서비스를 제공하기 위하여 노력하여야 한다. <개정 

# 4. Augmentation을 위한 Prompt 활용

- Retrieval 된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [31]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

# 5. 답변 생성

- RetrievalQA를 통해 LLM에 전달
    - RetrievalQA는 create_retrieval_chain으로 대체됨
    - 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음
- 하단의 dictionary_chain과 연계하여 사용

In [32]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [33]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [34]:
ai_message = qa_chain.invoke({"query": query})

In [35]:
ai_message

{'query': '의료기관에서 의료사고가 발생했을 때, 병원과 의료진의 법적 책임은 어떻게 되나요?',
 'result': '의료기관에서 의료사고가 발생했을 때, 병원과 의료진은 의료의 질을 높이고 환자에게 최선의 의료서비스를 제공할 의무를 위반한 경우 법적 책임을 질 수 있습니다. 또한, 진료 거부나 응급환자에 대한 적절한 처치를 하지 않은 경우에도 법적 책임이 따를 수 있습니다. 구체적인 법적 책임은 의료법 및 관련 법령에 따라 달라질 수 있습니다.'}

# 6. Retrieval을 위한 keyword 사전 활용

- Knowledge Base에서 사용되는 keyword를 활용하여 사용자 질문 수정
- LangChain Expression Language (LCEL)을 활용한 Chain 연계

In [36]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 의료법
dictionary = [
    "의료기록의 관리와 보호를 나타내는 표현 -> 의료기록 관리",
    "의료사고를 나타내는 표현 -> 의료사고",
    "의료진의 법적 책임을 나타내는 표현 -> 의료진 법적 책임",
    "환자의 권리를 나타내는 표현 -> 환자 권리",
    "의료정보의 보호를 나타내는 표현 -> 의료정보 보호"
]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요.
    사전: {dictionary}

    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [37]:
new_question = dictionary_chain.invoke({"question": query})

In [38]:
new_question

'의료기관에서 의료사고가 발생했을 때, 병원과 의료진 법적 책임은 어떻게 되나요?'

In [39]:
law_chain = {"query": dictionary_chain} | qa_chain

In [40]:
ai_response = law_chain.invoke({"question": query})

In [41]:
ai_response

{'query': '의료기관에서 의료사고가 발생했을 때, 병원과 의료진 법적 책임은 어떻게 되나요?',
 'result': '의료기관에서 의료사고가 발생했을 경우, 병원과 의료진은 의료법 및 관련 법령에 따라 법적 책임을 지게 됩니다. 구체적으로, 의료인과 의료기관의 장은 환자에게 최선의 의료서비스를 제공할 의무가 있으며, 이를 위반하여 의료사고가 발생하면 법적 처벌을 받을 수 있습니다. 또한, 의료기관의 의료용 시설, 기재, 약품 등을 점거하거나 파괴하는 행위도 법적으로 금지되어 있습니다.'}